<a href="https://colab.research.google.com/github/mishraShraddha/LGMVIP-Data-science/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b>Fake News Detection</b> <br>


<b>Dataset</b>: https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset
<br>
<b><U>About the Dataset:</U></b>

<b>Id</b>: unique id for a news article<br>
<b>Title</b>: the title of a news article<br>
<b>text</b>: the text of the article; could be incomplete<br>
<b>Subject</b>: News subject<br>
<b>Date</b>: published date <br>
<b>class</b>: a label that marks whether the news article is real or fake:<br>
       0: Fake <br>
       1: Real 

**Importing** **Required** **Libraries**

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

**Reading** **the** **Datasets**

In [2]:
df_fake=pd.read_csv("Fake.csv")
df_real=pd.read_csv("True.csv")
df_fake.head(10)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


In [3]:
df_real.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


Inserting 1 column class to dataframe to categorize real and fake news

In [4]:
df_fake["class"]= 0
df_real["class"]= 1

In [5]:
df_fake.shape,df_real.shape

((23481, 5), (21417, 5))

Droping last 10 rows and save it for manual testing

In [6]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
  df_fake.drop([i], axis=0, inplace=True)

df_real_manual_testing = df_real.tail(10)
for i in range(21416,21406,-1):
  df_real.drop([i],axis=0, inplace=True)

df_fake.shape,df_real.shape


((23471, 5), (21407, 5))

In [7]:
df_manual_testing=pd.concat([df_fake_manual_testing,df_real_manual_testing],axis=0)
df_manual_testing.to_csv("manual_testing.csv")

Merging the real and fake dataset

In [8]:
df_merge=pd.concat([df_fake,df_real],axis=0)
df_merge.head(10)


,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


Droping column date , subject and title which was not necessary .

In [9]:
df=df_merge.drop(["date","subject","title"],axis=1)
df.head(10)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


Shuffling the data 

In [10]:
df=df.sample(frac=1)
df.head(10)

,text,class
1177,WASHINGTON (Reuters) - U.S. President Donald T...,1
4615,WASHINGTON (Reuters) - The U.S. House voted on...,1
7884,NEW YORK (Reuters) - U.S. Republican presiden...,1
13100,"Finally, something else about those lawyers. I...",0
11668,"JOHANNESBURG (Reuters) - Cyril Ramaphosa, the ...",1
18215,RIYADH (Reuters) - Saudi Arabia said on Thursd...,1
2163,WASHINGTON (Reuters) - Mexico’s Economy Minist...,1
1346,"During the presidential campaign, Donald Trump...",0
13178,Thank you Lou Dobbs! He s been dead on this en...,0
13327,SEOUL/MOSCOW (Reuters) - North Korea s latest ...,1


In [11]:
df.isnull().sum()

text     0
class    0
dtype: int64

Cleaning the data 

In [12]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [13]:
df["text"] = df["text"].apply(wordopt)

In [14]:
df.head(10)

,text,class
1177,washington reuters u s president donald t...,1
4615,washington reuters the u s house voted on...,1
7884,new york reuters u s republican presiden...,1
13100,finally something else about those lawyers i...,0
11668,johannesburg reuters cyril ramaphosa the ...,1
18215,riyadh reuters saudi arabia said on thursd...,1
2163,washington reuters mexico s economy minist...,1
1346,during the presidential campaign donald trump...,0
13178,thank you lou dobbs he s been dead on this en...,0
13327,seoul moscow reuters north korea s latest ...,1


Splitting data into training and test data

In [15]:
x = df["text"]
y = df["class"]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.35)

Convert text to vectors

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

**1**.**Logistic** **Regression**

In [19]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
pred_lr=LR.predict(xv_test)
LR.score(xv_test, y_test)

0.9852941176470589

In [21]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      8162
           1       0.98      0.99      0.98      7546

    accuracy                           0.99     15708
   macro avg       0.99      0.99      0.99     15708
weighted avg       0.99      0.99      0.99     15708



**2**.**Decision** **Tree** **Classification**

In [22]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [23]:
pred_dt = DT.predict(xv_test)
DT.score(xv_test, y_test)

0.9971988795518207

In [24]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8162
           1       1.00      1.00      1.00      7546

    accuracy                           1.00     15708
   macro avg       1.00      1.00      1.00     15708
weighted avg       1.00      1.00      1.00     15708



**3**.**Gradient** **Boosting** **Classifier**

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [26]:
pred_gbc = GBC.predict(xv_test)

In [27]:
GBC.score(xv_test, y_test)

0.9964349376114082

In [28]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      8162
           1       0.99      1.00      1.00      7546

    accuracy                           1.00     15708
   macro avg       1.00      1.00      1.00     15708
weighted avg       1.00      1.00      1.00     15708



**4**.**Random** **Forest** **Classifier**

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [31]:
pred_rfc = RFC.predict(xv_test)

In [32]:
RFC.score(xv_test, y_test)

0.985039470333588

In [33]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      8162
           1       0.98      0.98      0.98      7546

    accuracy                           0.99     15708
   macro avg       0.99      0.98      0.99     15708
weighted avg       0.99      0.99      0.99     15708



**Model** **Testing**

In [34]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Real News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [35]:
news = str(input())
manual_testing(news)

21st Century Wire says This week, the historic international Iranian Nuclear Deal was punctuated by a two-way prisoner swap between Washington and Tehran, but it didn t end quite the way everyone expected. On the Iranian side, one of the U.S. citizens who was detained in Iran, Nosratollah Khosravi-Roodsari, has stayed in Iran, but on the U.S. side   all 7 of the Iranians held in U.S. prisons DID NOT show up to their flight to Geneva for the prisoner exchange   with at least 3 electing to stay in the U.S  TEHRAN SIDE: In Iran, 5 U.S. prisoners were released, with 4 of them making their way to Germany via Switzerland.Will Robinson Daily MailNone of the Iranians freed in the prisoner swap have returned home and could still be in the United States, it has been reported.The seven former inmates, who were released as part of a deal with the Islamic republic, did not show up to get a flight to Geneva, Switzerland, where the exchange was set to take place on Sunday.Three of the Iranians have d

In [37]:
news = str(input())
manual_testing(news)

LONDON (Reuters) - LexisNexis, a provider of legal, regulatory and business information, said on Tuesday it had withdrawn two products from the Chinese market in March this year after it was asked to remove some content.  The issue of academic freedom in China hit the headlines this week after the leading British academic publisher,  


LR Prediction: Real News 
DT Prediction: Real News 
GBC Prediction: Real News 
RFC Prediction: Real News


**Thank** **You**!!